In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import openpyxl
from rdkit.Chem.Draw import IPythonConsole, MolsToGridImage

#Show mols in dataframes
from rdkit.Chem import PandasTools
from rdkit import Chem
from rdkit.Chem.Draw import MolsToGridImage
from IPython.core.display import HTML
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
from rdkit.Chem import rdDepictor
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
import os
from scipy.spatial.distance import pdist, squareform
import sys

In [2]:
TOP = os.getcwd().replace('notebooks', '')
raw_dir = TOP + 'data/raw/'
interim_dir = TOP + 'data/interim/'
external_dir = TOP + 'data/external/'
figures_dir = TOP + 'reports/figures/'


In [3]:
import sqlalchemy as sa

In [5]:
#engine = sa.create_engine(f"""mysql+pymysql://{username}:{password}@{host}""")


In [6]:
#engine.execute("CREATE DATABASE toxval94") #create db
#engine.execute("USE toxval94")

In [7]:
toxvaldb = sa.create_engine(f"""mysql+pymysql://{username}:{password}@{host}/toxval94""")

In [8]:
toxvaldb.table_names()

/tmp/ipykernel_745407/164040344.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  toxvaldb.table_names()


['inhalation', 'irritation', 'oral_RDT', 'oral_RDT_dtxsid', 'pfas_data']

In [9]:

oral_df = pd.read_sql('''select dtxsid, source, subsource, risk_assessment_class,toxval_type, toxval_numeric,toxval_units_converted, toxval_units, study_type,source_source_id from oral_RDT ''', toxvaldb)

In [10]:
oral_df.shape

(99406, 10)

In [11]:
oral_df.toxval_type.unique()

array(['NOAEL', 'LOAEL', 'LEL', 'NOEL', 'LOEL', 'NOAEC', 'LOAEC', 'NEL',
       'NOEC', 'LOEC'], dtype=object)

In [14]:
oral_df.toxval_units.unique()

array(['mg/kg-day'], dtype=object)

In [15]:
def f10(x):
    return x.quantile(0.1)

In [16]:
oral_10pte = oral_df.groupby(['dtxsid', 'study_type', 'toxval_type']).agg({'toxval_numeric': f10})

In [17]:
oral_10pte = oral_10pte.reset_index()

In [18]:
res_1 = {}
for i, group in oral_10pte.groupby('dtxsid'):
    subset = group[['toxval_type', 'toxval_numeric']]
    a = list(subset[subset['toxval_type'].isin(['NOAEL', 'NOEC', 'NOEL', 'NOAEC','NEL'])]['toxval_numeric'])
    
    b = list(subset[subset['toxval_type'].isin(['LOAEL', 'LOEC', 'LOEL', 'LOAEC','LEL'])]['toxval_numeric'])
    b = [e/10 for e in b]
    
    if len(a) >0:
        res_1[i] = np.percentile(a, 10)
    elif len(b) > 0:
        res_1[i] = np.percentile(b, 10)

In [19]:
S1 = pd.DataFrame(list(res_1.items()), columns = ['dtxsid', '10ptile_POD'] )
S1

,dtxsid,10ptile_POD
0,DTXSID001004378,150.00000
1,DTXSID00100498,250.00000
2,DTXSID001005033,280.00000
3,DTXSID001006300,550.00000
4,DTXSID00100756,1000.00000
...,...,...
7631,DTXSID9098220,300.00000
7632,DTXSID90990466,150.00000
7633,DTXSID90998205,200.00000
7634,DTXSID9099876,1005.70000


In [20]:
oral_10pte =S1.loc[S1['dtxsid'] != 'NODTXSID']

oral_10pte

,dtxsid,10ptile_POD
0,DTXSID001004378,150.0
1,DTXSID00100498,250.0
2,DTXSID001005033,280.0
3,DTXSID001006300,550.0
4,DTXSID00100756,1000.0
...,...,...
7630,DTXSID9098147,450.0
7631,DTXSID9098220,300.0
7632,DTXSID90990466,150.0
7633,DTXSID90998205,200.0


In [21]:
ids = pd.read_sql('''select * from oral_RDT_dtxsid''', toxvaldb)

In [22]:
ids.columns

Index(['index', 'Query', 'Found_By', 'DSSTox_Substance_Id',
       'DSSTox_Structure_Id', 'DSSTox_QC-Level', 'Substance_Name',
       'Substance_CASRN', 'Substance_Type', 'Substance_Note',
       'Structure_SMILES', 'Structure_InChI', 'Structure_InChIKey',
       'Structure_Formula', 'Structure_MolWt', 'Structure_SMILES_2D-QSAR',
       'DateModified'],
      dtype='object')

In [23]:
oral_10pte = pd.merge(oral_10pte, ids[['Query','DSSTox_Substance_Id','Substance_Name','Structure_MolWt', 'Structure_SMILES','Structure_SMILES_2D-QSAR']], left_on = 'dtxsid', right_on = 'Query', how = 'left')

In [24]:
oral_10pte_smi = oral_10pte.dropna(subset=['Structure_SMILES_2D-QSAR'])

In [25]:
oral_10pte_smi= oral_10pte_smi[['DSSTox_Substance_Id', 'Substance_Name',
       'Structure_MolWt', 'Structure_SMILES', 'Structure_SMILES_2D-QSAR','10ptile_POD']]

In [26]:
oral_10pte_smi['pPOD'] = oral_10pte_smi[['10ptile_POD', 'Structure_MolWt']].apply(lambda x: -np.log10(x['10ptile_POD']/x['Structure_MolWt']), axis = 1)

In [27]:
oral_10pte_smi.to_csv(interim_dir+'oral_10pte_smi_020723.csv')

In [28]:
oral_10pte_smi

,DSSTox_Substance_Id,Substance_Name,Structure_MolWt,Structure_SMILES,Structure_SMILES_2D-QSAR,10ptile_POD,pPOD
0,DTXSID001004378,4-tert-Butylcyclohexyl prop-2-enoate,210.317,CC(C)(C)C1CCC(CC1)OC(=O)C=C,CC(C)(C)C1CCC(CC1)OC(=O)C=C,150.00,0.146783
2,DTXSID001005033,"Bis(2-ethylhexyl) cyclohexane-1,4-dicarboxylate",396.612,CCCCC(CC)COC(=O)C1CCC(CC1)C(=O)OCC(CC)CCCC,CCCCC(CC)COC(=O)C1CCC(CC1)C(=O)OCC(CC)CCCC,280.00,0.151208
3,DTXSID001006300,"6-[(1-Hydroxy-2,4-dimethylheptylidene)amino]he...",420.591,OCCN(CCO)CCO.CCCC(C)CC(C)C(O)=NCCCCCC(O)=O,CCCC(C)CC(C)C(=O)NCCCCCC(O)=O,550.00,-0.116503
6,DTXSID001009930,Thialdine,163.300,C[C@H]1N[C@@H](C)S[C@@H](C)S1,CC1NC(C)SC(C)S1,9.30,1.244503
16,DTXSID001014636,1-(2-Hydroxy-3-sulphonatopropyl)pyridinium,217.240,OC(C[N+]1=CC=CC=C1)CS([O-])(=O)=O,OC(C[N+]1=CC=CC=C1)CS([O-])(=O)=O,1000.00,-0.663060
...,...,...,...,...,...,...,...
7627,DTXSID90968887,Methyl 3-sulfanylbutanoate,134.190,COC(=O)CC(C)S,COC(=O)CC(C)S,0.06,3.349569
7628,DTXSID90976780,3-[(2-Methylfuran-3-yl)sulfanyl]butan-2-one,184.250,CC(SC1=C(C)OC=C1)C(C)=O,CC(SC1=C(C)OC=C1)C(C)=O,3.76,1.690220
7629,DTXSID90978461,Heptane-2-thiol,132.270,CCCCCC(C)S,CCCCCC(C)S,0.06,3.343310
7632,DTXSID90990466,"3-Amino-N-(8-hydroxy-4,6-disulfonaphthalen-1-y...",438.430,NC1=CC=CC(=C1)C(O)=NC1=C2C(O)=CC(=CC2=C(C=C1)S...,NC1=CC=CC(=C1)C(O)=NC1=C2C(O)=CC(=CC2=C(C=C1)S...,150.00,0.465809
